In [22]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pyspark.sql import functions as F
import glob
from pyspark.sql.types import *

In [2]:
municipios_br = spark.read.csv("/home/juracy.santos/dados/lista_municipios_brasil.csv",sep=",",header=True,inferSchema=False)

In [3]:
municipios_br.count()

5571

In [4]:
municipios_br = municipios_br.filter(F.col("Estado") != "9999999")

In [5]:
municipios_br.count()

5570

In [6]:
municipios_br.select("codigo").distinct().count()

5570

In [7]:
municipios_br.select("Estado").distinct().count()

27

In [8]:
municipios_br.select("nome_municipio","Estado").distinct().count()

5570

In [9]:
municipios_br.show(3)

+-------+-------------------+---------+---+------------+--------+---------+--------+
| codigo|     nome_municipio|Codigo_UF| UF|      Estado|Latitude|Longitude|codigo_6|
+-------+-------------------+---------+---+------------+--------+---------+--------+
|5200050|    Abadia de Goiás|       52| GO|       Goiás|-16.7573| -49.4412|  520005|
|3100104|Abadia dos Dourados|       31| MG|Minas Gerais|-18.4831| -47.3916|  310010|
|5200100|          Abadiânia|       52| GO|       Goiás| -16.197| -48.7057|  520010|
+-------+-------------------+---------+---+------------+--------+---------+--------+
only showing top 3 rows



In [10]:
municipios_br = municipios_br.withColumn("country_concept_id",F.lit("41892130") )
municipios_br = municipios_br.withColumn("state",F.col("UF") )
municipios_br = municipios_br.withColumn("city",F.col("nome_municipio") )
municipios_br = municipios_br.withColumn("country_source_value",F.lit("Brazil") )
municipios_br = municipios_br.withColumn("location_id",F.col("codigo") )
municipios_br = municipios_br.withColumn("location_source_value",F.col("codigo") )

In [11]:
municipios_br.printSchema()

root
 |-- codigo: string (nullable = true)
 |-- nome_municipio: string (nullable = true)
 |-- Codigo_UF: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- codigo_6: string (nullable = true)
 |-- country_concept_id: string (nullable = false)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country_source_value: string (nullable = false)
 |-- location_id: string (nullable = true)
 |-- location_source_value: string (nullable = true)



In [12]:
regioes  = spark.read.csv("/home/juracy.santos/dados/mapeamento_id_nome_regiao.csv",sep=";",header=True,inferSchema=False)

In [13]:
regioes.show()

+---------+------------+
|regiao_id| nome_regiao|
+---------+------------+
|        1|       Norte|
|        2|    Nordeste|
|        3|     Sudeste|
|        4|Centro-Oeste|
|        5|         sul|
+---------+------------+



In [14]:
municipios_br = municipios_br.withColumn("regiao_id",F.substring("codigo",1,1))

In [15]:
municipios_br_regiao = municipios_br.join(regioes,['regiao_id'],'left')

In [16]:
municipios_br_regiao.count()

5570

In [17]:
municipios_br_regiao = municipios_br_regiao.withColumn("county",F.col("nome_regiao") )

In [18]:
location_table = municipios_br_regiao.select("location_id","city","state","location_source_value","country_concept_id","county","country_source_value")

In [19]:
location_table.count()

5570

In [25]:
location_table = location_table.withColumn("location_id",F.col("location_id").cast(IntegerType()))
location_table = location_table.withColumn("country_concept_id",F.col("country_concept_id").cast(IntegerType()))

In [27]:
location_table.write.parquet("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_location.parquet")

In [20]:
location_table.toPandas().to_csv("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_location.csv",sep=",",index=False)

In [21]:
location_table.printSchema()

root
 |-- location_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- location_source_value: string (nullable = true)
 |-- country_concept_id: string (nullable = false)
 |-- county: string (nullable = true)
 |-- country_source_value: string (nullable = false)



In [63]:
location_table.show()

+-----------+-------------------+--------------+---------------------+------------------+------------+--------------------+
|location_id|               city|         state|location_source_value|country_concept_id|      county|country_source_value|
+-----------+-------------------+--------------+---------------------+------------------+------------+--------------------+
|    5200050|    Abadia de Goiás|         Goiás|              5200050|          41892130|         sul|              Brazil|
|    3100104|Abadia dos Dourados|  Minas Gerais|              3100104|          41892130|     Sudeste|              Brazil|
|    5200100|          Abadiânia|         Goiás|              5200100|          41892130|         sul|              Brazil|
|    3100203|             Abaeté|  Minas Gerais|              3100203|          41892130|     Sudeste|              Brazil|
|    1500107|         Abaetetuba|          Pará|              1500107|          41892130|       Norte|              Brazil|
|    230